In [1]:
from Bio import SeqIO
from Bio.Align.Applications import ClustalOmegaCommandline
from Bio import Phylo, AlignIO
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
import os
import re
import matplotlib.pyplot as plt

In [ ]:
# Função para extrair sequências do arquivo FASTA
def extract_sequences_from_fasta(fasta_file):
    sequences = []
    id_counts = {}
    
    for record in SeqIO.parse(fasta_file, "fasta"):
        description = record.description
        
        # Extraindo nome completo da espécie e estirpe (se presente)
        species_info = re.search(r'(\S+\s\S+.*?strain\s\S+|\S+\s\S+)', description)
        species_name = species_info.group(0) if species_info else 'Unknown_Species'
        
        if species_name in id_counts:
            id_counts[species_name] += 1
            sequence_id = f"{species_name}_hsps{id_counts[species_name]}"
        else:
            id_counts[species_name] = 1
            sequence_id = species_name
        
        # Gerando o cabeçalho corretamente formatado (apenas nome da espécie e estirpe)
        header = f">{species_name}"
        sequence = f"{header}\n{str(record.seq)}\n"
        sequences.append(sequence)

    return sequences

# Função para escrever o arquivo FASTA com as sequências (mantida igual)
def write_fasta(sequences, output_file):
    with open(output_file, 'w') as fasta_file:
        fasta_file.writelines(sequences)

# Função para calcular a distância e construir a árvore filogenética (mantida igual)
def build_phylogenetic_tree(gene_name, sequences, output_dir):
    calculator = DistanceCalculator("identity")
    dm = calculator.get_distance(sequences)
    constructor = DistanceTreeConstructor()
    tree = constructor.upgma(dm)

    modify_tree_labels(tree)

    plot_tree(tree, output_dir, gene_name)
    print(f"Phylogenetic tree for {gene_name} saved to {os.path.join(output_dir, f'{gene_name}_phylogenetic_tree.png')}")

# Função para modificar os rótulos da árvore (mantida igual)
def modify_tree_labels(tree):
    for clade in tree.find_clades():
        if clade.name:
            match = re.search(r'(\S+\s\S+.*?strain\s\S+|\S+\s\S+)', clade.name)
            if match:
                clade.name = match.group(0)

# Função para desenhar a árvore filogenética (mantida igual)
def plot_tree(tree, output_dir, gene_name):
    fig = plt.figure(figsize=(35, 15))
    ax = fig.add_subplot(1, 1, 1)
    Phylo.draw(tree, do_show=False, axes=ax)
    
    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_fontsize(18)
    for text in ax.findobj(match=plt.Text):
        text.set_fontsize(22)
    
    plt.title(f'{gene_name} - Phylogenetic Tree', fontsize=30)

    tree_img_path = os.path.join(output_dir, f'{gene_name}_phylogenetic_tree.png')
    plt.savefig(tree_img_path, bbox_inches='tight')
    plt.close()

# Função para processar o gene: ler FASTA, gerar alinhamento e construir a árvore
def process_gene(gene_name, fasta_file, output_dir):
    print(f"Processing gene: {gene_name}")

    # Passo 1: Extrair sequências do arquivo FASTA
    sequences = extract_sequences_from_fasta(fasta_file)
    fasta_file_path = os.path.join(output_dir, f"{gene_name}_aligned.fasta")
    write_fasta(sequences, fasta_file_path)
    print(f"FASTA file for {gene_name} saved to {fasta_file_path}")

    # Passo 2: Alinhar as sequências com o Clustal Omega
    clustalomega_exe_path = r"C:\Program Files\clustal-omega-1.2.2-win64\clustalo.exe"
    clustalomega_cline = ClustalOmegaCommandline(clustalomega_exe_path, infile=fasta_file_path, outfile=fasta_file_path, force=True)
    clustalomega_cline()
    print(f"Alignment for {gene_name} saved to {fasta_file_path}")

    # Passo 3: Carregar o alinhamento e construir a árvore filogenética
    aligned_file_path = AlignIO.read(fasta_file_path, "fasta")
    build_phylogenetic_tree(gene_name, aligned_file_path, output_dir)

# Diretórios de entrada e saída
blast_results_dir = "blast_results"  # Pasta onde os ficheiros FASTA estão armazenados
output_dir = "output"  # O diretório onde as saídas serão salvas
os.makedirs(output_dir, exist_ok=True)

# Lista de genes a processar
genes = ["glgA", "glgB", "glgD", "93290188"]

# Processar cada gene
for gene in genes:
    fasta_file = os.path.join(blast_results_dir, f"{gene}_blast.fasta")  # Caminho para o ficheiro FASTA na pasta "blast_results"
    if os.path.exists(fasta_file):
        process_gene(gene, fasta_file, output_dir)
    else:
        print(f"File {fasta_file} not found. Skipping gene {gene}.")

In [15]:
from Bio.Align.Applications import ClustalOmegaCommandline
from Bio import Phylo, AlignIO, SeqIO
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
import os

# Lista de genes
genes = ["glgA", "glgB", "glgD", "93290188"]  # Substitua pelos seus genes

# Caminho para os arquivos FASTA dos resultados do BLAST
blast_files = [f"blast_results/{gene}_blast.fasta" for gene in genes]

# Cria a pasta "alinhamento_e_filogenia" se ela não existir
output_folder = "alinhamento_e_filogenia"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for gene, blast_file in zip(genes, blast_files):
    # Verifica se o arquivo FASTA do BLAST existe
    if not os.path.exists(blast_file):
        print(f"Arquivo FASTA do BLAST não encontrado para {gene}: {blast_file}. Pulando...")
        continue

    # Carrega as sequências do arquivo FASTA
    sequences = list(SeqIO.parse(blast_file, "fasta"))

    # Garante que os IDs das sequências sejam únicos
    unique_sequences = []
    id_counts = {}
    for seq in sequences:
        if seq.id in id_counts:
            id_counts[seq.id] += 1
            seq.id = f"{seq.id}_dup{id_counts[seq.id]}"  # Adiciona um sufixo único
        else:
            id_counts[seq.id] = 1
        unique_sequences.append(seq)

    # Salva as sequências em um arquivo FASTA temporário para o alinhamento
    fasta_file_path = os.path.join(output_folder, f"{gene}_sequences.fasta")
    SeqIO.write(unique_sequences, fasta_file_path, "fasta")

    # Alinhar as sequências com o Clustal Omega
    clustalomega_exe_path = r"C:\Program Files\clustal-omega-1.2.2-win64\clustalo.exe"
    aligned_file_path = os.path.join(output_folder, f"{gene}_aligned.fasta")  # Arquivo de saída para o alinhamento

    clustalomega_cline = ClustalOmegaCommandline(
        clustalomega_exe_path,
        infile=fasta_file_path,  # Arquivo de entrada
        outfile=aligned_file_path,  # Arquivo de saída
        force=True
    )
    clustalomega_cline()
    print(f"Alinhamento para {gene} salvo em {aligned_file_path}")

    # Lê as sequências alinhadas
    aligned_sequences = AlignIO.read(aligned_file_path, "fasta")

    # Calcula as distâncias para o gene atual
    calculator = DistanceCalculator("identity")
    dm = calculator.get_distance(aligned_sequences)

    # Constrói a árvore filogenética
    constructor = DistanceTreeConstructor()
    tree = constructor.upgma(dm)

    # Salva a árvore filogenética
    tree_file_path = os.path.join(output_folder, f"{gene}_phylogenetic_tree.newick")
    Phylo.write(tree, tree_file_path, "newick")
    print(f"Árvore filogenética para {gene} salva em {tree_file_path}")

Alinhamento para glgA salvo em alinhamento_e_filogenia\glgA_aligned.fasta
Árvore filogenética para glgA salva em alinhamento_e_filogenia\glgA_phylogenetic_tree.newick
Alinhamento para glgB salvo em alinhamento_e_filogenia\glgB_aligned.fasta
Árvore filogenética para glgB salva em alinhamento_e_filogenia\glgB_phylogenetic_tree.newick
Alinhamento para glgD salvo em alinhamento_e_filogenia\glgD_aligned.fasta
Árvore filogenética para glgD salva em alinhamento_e_filogenia\glgD_phylogenetic_tree.newick
Alinhamento para 93290188 salvo em alinhamento_e_filogenia\93290188_aligned.fasta
Árvore filogenética para 93290188 salva em alinhamento_e_filogenia\93290188_phylogenetic_tree.newick


In [ ]:
from Bio import Phylo
import os

# Caminho para a pasta "alinhamento_e_filogenia"
output_folder = "alinhamento_e_filogenia"

# Lista todos os arquivos Newick na pasta
newick_files = [f for f in os.listdir(output_folder) if f.endswith(".newick")]

# Visualiza cada árvore
for newick_file in newick_files:
    # Caminho completo para o arquivo Newick
    tree_file_path = os.path.join(output_folder, newick_file)
    
    # Carrega a árvore
    tree = Phylo.read(tree_file_path, "newick")
    
    # Exibe o nome do arquivo
    print(f"Visualizando árvore: {newick_file}")
    
    # Visualiza a árvore
    Phylo.draw_ascii(tree)

In [21]:
from Bio import Phylo
import matplotlib.pyplot as plt
import os

# Caminho para a pasta "alinhamento_e_filogenia"
output_folder = "alinhamento_e_filogenia"

# Lista todos os arquivos Newick na pasta
newick_files = [f for f in os.listdir(output_folder) if f.endswith(".newick")]

# Configurações do gráfico
plt.rcParams["font.size"] = 12  # Tamanho da fonte para os rótulos

# Processa cada arquivo Newick
for newick_file in newick_files:
    # Caminho completo para o arquivo Newick
    tree_file_path = os.path.join(output_folder, newick_file)
    
    # Extrai o nome do gene a partir do nome do arquivo
    gene_name = newick_file.split("_")[0]  # Assumindo que o nome do arquivo é "gene_phylogenetic_tree.newick"
    
    # Carrega a árvore
    tree = Phylo.read(tree_file_path, "newick")
    
    # Cria a figura e o eixo
    fig = plt.figure(figsize=(30, 10))
    ax = fig.add_subplot(1, 1, 1)
    
    # Desenha a árvore sem os nomes dos nós internos
    Phylo.draw(
        tree,
        do_show=False,
        axes=ax,
        show_confidence=False,  # Remove os valores de confiança (se houver)
        branch_labels=None,     # Remove os rótulos dos ramos
        label_func=lambda x: "" if x.name and x.name.startswith("Inner") else x.name  # Remove rótulos dos nós internos
    )
    
    # Adiciona um título
    plt.title(f"Árvore Filogenética - {gene_name}", fontsize=22)
    
    # Ajusta o tamanho da fonte dos rótulos das folhas
    for label in ax.texts:
        label.set_fontsize(16)
    
    # Salva a imagem da árvore
    tree_img = os.path.join(output_folder, f"{gene_name}_phylogenetic_tree.png")
    plt.savefig(tree_img, bbox_inches='tight', dpi=300)  # Salva em alta resolução
    plt.close()  # Fecha a figura para liberar memória
    
    print(f"Imagem da árvore salva em: {tree_img}")

Imagem da árvore salva em: alinhamento_e_filogenia\93290188_phylogenetic_tree.png
Imagem da árvore salva em: alinhamento_e_filogenia\glgA_phylogenetic_tree.png
Imagem da árvore salva em: alinhamento_e_filogenia\glgB_phylogenetic_tree.png
Imagem da árvore salva em: alinhamento_e_filogenia\glgD_phylogenetic_tree.png
